In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
%matplotlib inline
import os
import glob
#import cv2
from matplotlib import pyplot as plt
from keras import utils as np_utils
from keras.utils import to_categorical
from keras.optimizers import RMSprop
from keras.layers import LSTM, Dense, Dropout
from keras.layers import Dense, Input, Conv1D, Flatten, MaxPooling1D, Activation,Dropout,BatchNormalization
from keras.models import Sequential
import pandas as pd 
import numpy as np 
import keras

Using TensorFlow backend.


In [6]:
bengalichar = {"া" : 100,
               "ি" : 150,
               "ী" : 200,
               "ু" : 250,
               "ূ" : 300,
               "ৃ" : 350,
               "ে" : 400,
               "ৈ" : 450,
               "ো" : 500,
               "ৌ" : 550,
               "ং" : 600,
               "ঃ" : 650,
               "ঁ" : 700,
               "অ" : 1,
               "আ" : 2,
               "ই" : 3,
               "ঈ" : 4,
               "উ" : 5,
               "ঊ" : 6,
               "ঋ" : 7,
               "এ" : 8,
               "ঐ" : 9,
               "ও" : 10,
               "ঔ" : 11,
               "ক" : 21,
               "খ" : 22,
               "গ" : 23,
               "ঘ" : 24,
               "ঙ" : 25,
               "চ" : 26,
               "ছ" : 27,
               "জ" : 28,
               "ঝ" : 29,
               "ঞ" : 30,
               "ট" : 31,
               "ঠ" : 32,
               "ড" : 33,
               "ঢ" : 34,
               "ণ" : 35,
               "ত" : 36,
               "থ" : 37,
               "দ" : 38,
               "ধ" : 39,
               "ন" : 40,
               "প" : 41,
               "ফ" : 42,
               "ব" : 43,
               "ভ" : 44,
               "ম" : 45,
               "য" : 46,
               "র" : 47,
               "ল" : 48,
               "শ" : 49,
               "ষ" : 50,
               "স" : 51,
               "হ" : 52,
               "ড়" : 53,
               "ঢ়" : 54,
               "য়" : 55,
               "ৎ" : 56
              }

def to_label(y):
  y_label = []
  for i in y:
    l = list(i)
    val = 0
    for j in l:
      val = val + bengalichar[j]
    y_label.append(val)
  print("Labeled {}/{}".format(len(y_label), len(y)))
  return y_label

In [7]:
def get_key(path):
    # seperates the key of an image from the filepath
    key=path.split(sep=os.sep)[-1]
    return key

In [8]:
def get_data(paths_img, path_label=None):
  X = []
  data_dir = 'C:\Desktop\bengalidataset'
  paths_img = glob.glob(os.path.join(data_dir, paths_img,'*.png'))
  for i, path in enumerate(paths_img):
    img = cv2.imread(path)
    img = np.float32(img)
    img=cv2.resize(img,(128,128), interpolation=cv2.INTER_AREA)
    #removing blur
    gaussian = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.addWeighted(img, 1.5, gaussian, -0.5, 0, img)
    #plt.imshow(img)
    #plt.show()
    X.append(img)

    #display progress
    if (i==len(paths_img)-1):
      print('processed {}/{}'.format(i+1,len(paths_img)))
    
  X = np.array(X)
  if (path_label is None):
    return X
  else:
    #read labels
    path_label = os.path.join(data_dir, path_label)
    df = pd.read_csv(path_label) 
    df = df.set_index('filename')
    #get the labels corresponding to the images
    y = [df.loc[get_key(path)]['word'] for path in  paths_img] 
    y = to_label(y)
    y=to_categorical(y,1000)
    return X, y

In [9]:
X_train, y_train = get_data('training', 'training.csv')
X_test, y_test = get_data('test', 'test.csv')

FileNotFoundError: [Errno 2] File b'C:\\Desktop\x08engalidataset\\training.csv' does not exist: b'C:\\Desktop\x08engalidataset\\training.csv'

In [7]:
X_train.shape

NameError: name 'X_train' is not defined

In [0]:
X_train = X_train[:, :, :, 0]
X_train.shape

(360, 128, 128)

In [0]:
X_test = X_test[:, :, :, 0]
X_test.shape

(12, 128, 128)

In [0]:

batch_size = 128
input_shape = (128, 128)
nb_lstm_outputs = 30
nb_classes = 3
#Buliding RNN Model
model = Sequential()
model.add(LSTM(nb_lstm_outputs, input_shape=input_shape, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1000, activation='softmax'))
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 30)           19080     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128, 30)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               1112064   
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 1000)              513000    
Total params: 1,644,144
Trainable params: 1,644,144
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = model.fit(X_train, y_train, epochs=10, 
                    batch_size=batch_size,
                    validation_data = (X_test, y_test),
                    shuffle=True, verbose=1)

Train on 360 samples, validate on 12 samples
Epoch 1/10
360/360 [==============================] - 11s 32ms/step - loss: 6.8541 - acc: 0.0139 - val_loss: 6.7189 - val_acc: 0.0000e+00
Epoch 2/10
360/360 [==============================] - 9s 25ms/step - loss: 6.3984 - acc: 0.0333 - val_loss: 5.5092 - val_acc: 0.0833
Epoch 3/10
360/360 [==============================] - 9s 25ms/step - loss: 5.7523 - acc: 0.1028 - val_loss: 5.5565 - val_acc: 0.1667
Epoch 4/10
360/360 [==============================] - 9s 24ms/step - loss: 5.0994 - acc: 0.0944 - val_loss: 5.0112 - val_acc: 0.1667
Epoch 5/10
360/360 [==============================] - 9s 25ms/step - loss: 4.9839 - acc: 0.1194 - val_loss: 5.2032 - val_acc: 0.0833
Epoch 6/10
360/360 [==============================] - 9s 25ms/step - loss: 4.4902 - acc: 0.1056 - val_loss: 4.0329 - val_acc: 0.3333
Epoch 7/10
360/360 [==============================] - 9s 24ms/step - loss: 4.0463 - acc: 0.2139 - val_loss: 4.0484 - val_acc: 0.0000e+00
Epoch 8/10
360/

In [0]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20, 5), max_iter=150, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)



In [0]:
mlp.fit(X_train, y_train)

ValueError: ignored

In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape = (128, 128)))
model.add(Dense(512))
model.add(Dense(1000, activation='softmax'))
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
batch_size = 128
history = model.fit(X_test, y_test, epochs=10, 
                    batch_size=batch_size,
                    validation_data = (X_train, y_train),
                    shuffle=True, verbose=1)

ValueError: ignored

In [0]:
input_shape = (128, 128)
batch_size = 128
model = Sequential()
model.add(Conv1D(32, (5), input_shape=input_shape, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=6))
model.add(Flatten())
model.add(Dense(1000, activation='softmax'))
model.add(Dropout(0.2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [0]:
history = model.fit(X_train, y_train, 
            batch_size=batch_size, 
            epochs=10, 
            verbose=1, 
            validation_data=(X_test, y_test),
            shuffle=True
            )

Train on 360 samples, validate on 12 samples
Epoch 1/10
360/360 [==============================] - 2s 5ms/step - loss: 6.8791 - acc: 0.1028 - val_loss: 6.8250 - val_acc: 0.1667
Epoch 2/10
360/360 [==============================] - 0s 727us/step - loss: 6.7216 - acc: 0.2583 - val_loss: 6.7506 - val_acc: 0.3333
Epoch 3/10
360/360 [==============================] - 0s 741us/step - loss: 6.6103 - acc: 0.5000 - val_loss: 6.6642 - val_acc: 0.4167
Epoch 4/10
360/360 [==============================] - 0s 739us/step - loss: 6.5193 - acc: 0.6556 - val_loss: 6.5735 - val_acc: 0.5833
Epoch 5/10
360/360 [==============================] - 0s 740us/step - loss: 6.3854 - acc: 0.6833 - val_loss: 6.3927 - val_acc: 0.6667
Epoch 6/10
360/360 [==============================] - 0s 752us/step - loss: 6.2054 - acc: 0.7611 - val_loss: 6.2705 - val_acc: 0.7500
Epoch 7/10
360/360 [==============================] - 0s 770us/step - loss: 6.1726 - acc: 0.7667 - val_loss: 6.2394 - val_acc: 0.8333
Epoch 8/10
360/360 